In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

path = 'C:/Users/queenSSH/Desktop/학교/CUAI/cuai하계/최종데이터/'

naver = pd.read_csv(path + '네이버_웹툰.csv' , encoding='cp949')
daum = pd.read_csv(path + 'daum_final.csv', encoding = 'cp949')
lezhin = pd.read_csv(path + '레진코믹스_웹툰.csv', encoding='UTF8')
kakao = pd.read_csv(path + 'kakao_page_완결_인코딩.csv', encoding = 'cp949')



web_combined = pd.concat([naver, daum, lezhin, kakao], axis=0, ignore_index=True)
#web_combined = pd.concat([naver, daum], axis=0, ignore_index=True)

In [2]:
import sys
sys.path.append('../')
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
import krwordrank

#### 결측치 채우기 (장르로 대체)

In [3]:
story_list = []

for i in range(len(web_combined['story'])):    
    if type(web_combined['story'][i]) == float:
        web_combined['story'][i] = web_combined['genre'][i]
    
    else:       
        web_combined['story'][i].replace('\n','')
        web_combined['story'][i].replace('&lt;','')
        web_combined['story'][i].replace('&gt;','')
        
    story_list.append(web_combined['story'][i])
    

<ipython-input-3-2f234ecb45a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  web_combined['story'][i] = web_combined['genre'][i]


In [4]:
print(type(list(story_list[0:1])))

<class 'list'>


In [5]:
print(type(story_list))

<class 'list'>


In [6]:
print(story_list[4218:4219])

['공부밖에 모르는 쑥맥 안경현. 그의 앞에 자신의 여자 친구라고 주장하는 세명의 여자가 나타났다! 안경현을 차지하기 위해 한치도 물러설 생각이 없는 세 여자친구의 전쟁이 지금 시작된다!']


In [7]:
web_combined.head(4220)

,Unnamed: 0,id,title,author,day,genre,platform,story,age,rate,free,complete,thumbnail,web,website
0,0,0.0,마물전담부대,염손,0,"스토리, 판타지",네이버 웹툰,마법혁명으로 근대화를 이룬 판타지 세계.\n하지만 이러한 세계에서도 마물은 여전히 ...,0.0,9.11,2,1,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...,NaN
1,1,1.0,불릿,김도근,0,"스토리, 스포츠",네이버 웹툰,"축구부 에이스, 좋아하는 선배에게 고백하려 했는데... 에어소프트 게임 마스터가 된다?!",0.0,9.89,1,1,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...,NaN
2,2,2.0,더,공현곤,0,"스토리, 스릴러",네이버 웹툰,수천년 전 조상들의 경주로 정해진 12인의 운명. 자리를 지키려는 1등과 바꾸려는 ...,15.0,9.94,1,1,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...,NaN
3,3,3.0,각자의,오묘,0,"스토리, 로맨스",네이버 웹툰,헤어졌지만 헤어지지 않은 사이\n솔직하지 못한 우리들 어떻게 될까요?,0.0,9.97,1,1,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...,NaN
4,4,4.0,노선도,효기,0,"스토리, 스릴러",네이버 웹툰,암호화폐 투자 실패로 모든 걸 잃은 평범한 40대 회사원 '권한수'.\n절망한 그의...,15.0,9.82,1,1,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4215,1985,1985.0,스킵과 로퍼,타카마츠 미사키,NaN,드라마,레진코믹스,"고등학교 1학년 봄, 지방에서 진학을 위해 도쿄로! 미래 설계는 완벽하지만 살짝 엉...",15.0,NaN,유료,연재중,https://ccdn.lezhin.com/v2/comics/672732043418...,NaN,https://www.lezhin.com/ko/comic/skip_to_loafers
4216,1986,1986.0,기레기,너믿설,NaN,드라마,레진코믹스,"쓰레기 기자, 순수한 살인마를 만나다!! 병구는 강경하고 교묘한 취재로 남달리 자극...",15.0,NaN,유료,연재중,https://ccdn.lezhin.com/v2/comics/618477127178...,NaN,https://www.lezhin.com/ko/comic/trash_reporter
4217,1987,1987.0,독일 만화,맨오브피스,NaN,일상,레진코믹스,일상,15.0,NaN,유료,연재중,https://ccdn.lezhin.com/v2/comics/569266149785...,NaN,https://www.lezhin.com/ko/comic/germantoon
4218,1988,1988.0,나와 그녀와 그녀와 그녀의 건전하지 못한 관계,"서영일, 최지인",NaN,학원,레진코믹스,공부밖에 모르는 쑥맥 안경현. 그의 앞에 자신의 여자 친구라고 주장하는 세명의 여자...,0.0,NaN,유료,연재중,https://ccdn.lezhin.com/v2/comics/392/images/t...,NaN,https://www.lezhin.com/ko/comic/threegirlfriends


In [8]:
key_wordlist= [] 
wordrank_extractor = KRWordRank(
    min_count = 1, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 2, # 단어의 최대 길이
    verbose = True
    )

beta = 0.8    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(story_list[4218:4219], beta, max_iter)

scan vocabs ... 
num vocabs = 53
done = 10


In [9]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
    print('%8s:\t%.4f' % (word, r))

      여자:	1.4046
     세명의:	1.0778
     물러설:	1.0688
     한치도:	1.0277
     모르는:	0.9447
     생각이:	0.9418
    안경현.:	0.9261
     전쟁이:	0.9003
      쑥맥:	0.8966
     자신의:	0.8621
      앞에:	0.8446
      지금:	0.7143
      위해:	0.6799
      없는:	0.6799
    차지하기:	0.6727
    공부밖에:	0.6557
   시작된다!:	0.6545
      그의:	0.6246
    친구라고:	0.5929
   나타났다!:	0.5929
    주장하는:	0.4880


In [10]:
# key_wordlist= [] 
# wordrank_extractor = KRWordRank(
#     min_count = 1, # 단어의 최소 출현 빈도수 (그래프 생성 시)
#     max_length = 2, # 단어의 최대 길이
#     verbose = True
#     )

# beta = 0.8    # PageRank의 decaying factor beta
# max_iter = 10

# for i in range(len(story_list)):
#     keywords, rank, graph = wordrank_extractor.extract(story_list[i:i+1], beta, max_iter)
    
#     for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:3]:
#         print('%8s:\t%.4f' % (word, r))
#         key_wordlist.append(word)
        
    
    
    

In [11]:
print(keywords)

{'여자': 1.4046030657534878, '세명의': 1.0778299512535376, '물러설': 1.068755456, '한치도': 1.0277301603083757, '모르는': 0.9446513199139376, '생각이': 0.9417873385500181, '안경현.': 0.9261150069633827, '전쟁이': 0.9003402757862522, '쑥맥': 0.8966316133545333, '자신의': 0.8620864721209471, '앞에': 0.8446442255231701, '지금': 0.7143156735999999, '위해': 0.6799379455999999, '없는': 0.6799379455999999, '차지하기': 0.6727212086613332, '공부밖에': 0.6556911100282652, '시작된다!': 0.6544777003374516, '그의': 0.6246250688195727, '친구라고': 0.5929309620825227, '나타났다!': 0.5929309620825227, '주장하는': 0.4880390752548682}


In [12]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
    print('%8s:\t%.4f' % (word, r))

      여자:	1.4046
     세명의:	1.0778
     물러설:	1.0688
     한치도:	1.0277
     모르는:	0.9447
     생각이:	0.9418
    안경현.:	0.9261
     전쟁이:	0.9003
      쑥맥:	0.8966
     자신의:	0.8621
      앞에:	0.8446
      지금:	0.7143
      위해:	0.6799
      없는:	0.6799
    차지하기:	0.6727
    공부밖에:	0.6557
   시작된다!:	0.6545
      그의:	0.6246
    친구라고:	0.5929
   나타났다!:	0.5929
    주장하는:	0.4880
